# UNIVERSIDAD NACIONAL DE HURLINGHAM
## Inst. de Tecnología e Ingeniería
### PARSEO Y GENERACIÓN DE CÓDIGO
#### Prof: Mag. Ing. Pablo Pandolfo
#### Examen Parcial - Noviembre 2025

* **ALUMNO**:
* **DNI**:

#### NOTA: EL EXAMEN ESCRITO ES UN DOCUMENTO DE GRAN IMPORTANCIA PARA LA EVALUACIÓN DE LOS CONOCIMIENTOS ADQUIRIDOS, POR LO TANTO, SE SOLICITA LEER ATENTAMENTE LO SIGUIENTE

* Responda claramente cada punto, detallando con la mayor precisión posible lo solicitado.
* Sea prolijo y ordenado en el desarrollo de los temas.
* Sea cuidadoso con las faltas de ortografía y sus oraciones.
* No desarrollar el examen en lápiz.
* Aprobación del examen: Con nota mayor o igual a 4 (cuatro)
* Condiciones de aprobación: 60%
* Duración de examen: 3 horas.

1. **[2.5 puntos]** Diséñese el diagrama de transiciones para un scanner del lenguaje cuyas palabras verifican las siguientes restricciones:
    * Una cadena comando es el nombre del comando seguido de uno o más espacios en blanco, y a continuación una lista de parámetros que puede ser vacía.
    * Un comando es una secuencia no vacía de cadenas comando separadas por un delimitador.
    * Un delimitador es un elemento del conjunto {. , ;}.
    * Un nombre comienza con una letra y sigue con cero o más letras o dígitos. La lista de parámetros es una secuencia de uno o más nombres separados por uno o más espacios en blanco

$$COM  \rightarrow CAD \mid CAD \; DEL \; COM$$
$$DEL  \rightarrow . \mid ;$$
$$CAD  \rightarrow ID \mid ID \; LPAR$$
$$LPAR \rightarrow PAR \mid PAR \; LPAR$$
$$PAR  \rightarrow ID$$
$$ID   \rightarrow L \mid L \; A$$
$$A    \rightarrow L \mid D \mid L \; A \mid D \; A$$
$$L    \rightarrow a \mid ... \mid z \mid A \mid ... \mid Z$$
$$D    \rightarrow 0 \mid ... \mid 9$$

* **DEL** = (. | ;)
* **ID** = L (L | D)*

In [58]:
from mermaid import Mermaid
Mermaid('''
flowchart LR
    q0((>q0))--.|;-->q1(((q1 DEL)));
    q0--L-->q2(((q2 ID)));
    q2--L|D-->q2;
''')

2. **[1 punto]** Analícese la entrada: **run test1 test2;build main;deploy**

1. Estado=q0
2. Entrada=**r**
3. Estado=q2
4. Entrada=**u**
5. Estado=q2
6. Entrada=**n**
7. Estado=q2
8. Token=**ID**
9. Lexema=**run**
10. ...

3. **[1.5 puntos]** Impleméntese scanner en Ply

In [59]:
%%file comando.py
import ply.lex as lex

# definir tokens
tokens  = ('DEL', 'ID')

# definir patrones
t_DEL    = r'(. | ;)'
t_ID     = r'[a-zA-Z_][a-zA-Z_0-9]*'
t_ignore = " \t"

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count("\n")
    
def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)

# construir scanner
lexer = lex.lex()
lexer.input('run test1 test2;build main;deploy')
while 1:
    tok = lexer.token()
    if not tok: break
    print(tok)

Overwriting comando.py


In [60]:
!python comando.py

LexToken(ID,'run',1,0)
LexToken(ID,'test1',1,4)
LexToken(ID,'test2',1,10)
LexToken(DEL,';',1,15)
LexToken(ID,'build',1,16)
LexToken(ID,'main',1,22)
LexToken(DEL,';',1,26)
LexToken(ID,'deploy',1,27)


4. **[2.5 puntos]** Compruébese si la siguiente GIC es LL(1), mostrando los conjuntos PRIM, SIG y PRED.

$$P \rightarrow \{L$$
$$L \rightarrow \} \mid S;L$$
$$S \rightarrow V = E$$
$$V \rightarrow a \mid b \mid c$$
$$E \rightarrow V O$$
$$O \rightarrow + V \mid - V \mid V$$

* PRIM(P) = {{}
* PRIM(L) = {}, a, b, c}
* PRIM(S) = {a, b, c}
* PRIM(V) = {a, b, c}
* PRIM(E) = {a, b, c}
* PRIM(O) = {+, -, a, b, c}
* SIG(P) = {\$}
* SIG(L) = {\$}
* SIG(S) = {;}
* SIG(V) = {=, +, -, ;, a, b, c}
* SIG(E) = {;}
* SIG(O) = {;}
* PRED(P -> {L) = {{}
* PRED(L -> }) = {}}
* PRED(L -> S;L) = {a, b, c}
* PRED(S -> V = E) = {a, b, c}
* PRED(V -> a) = {a}
* PRED(V -> b) = {b}
* PRED(V -> c) = {c}
* PRED(E -> VO) = {a, b, c}
* PRED(O -> +V) = {+}
* PRED(O -> -V) = {-}
* PRED(O -> V) = {a, b, c}

|    | {       | }      | a          | b          | c          | +       | -       | ;     | =     | $     |
| -- | --      | --     | --         | --         | --         | --      | --      | --    | --    | --    |
| P  | P -> {L | error  | error      | error      | error      | error   | error   | error | error | error |
| L  | error   | L -> } | L -> S;L   | L -> S;L   | L -> S;L   | error   | error   | error | error | error |
| S  | error   | error  | S -> V = E | S -> V = E | S -> V = E | error   | error   | error | error | error |
| V  | error   | error  | V -> a     | V -> b     | V -> c     | error   | error   | error | error | error |
| E  | error   | error  | E -> VO    | E -> VO    | E -> VO    | error   | error   | error | error | error |
| O  | error   | error  | O -> V     | O -> V     | O -> V     | O -> +V | O -> -V | error | error | error |

5. **[1 punto]** Muéstrese los movimientos realizados por el ASDP LL(1) con la entrada: **{a = a + b; b = c}**

| Pila   | Cadena         | Acción     |
| --     | --             | --         |
| \$P     | {a=a+b;b=c}\$ | P -> {L    |
| \$L{    | {a=a+b;b=c}\$ | emp({)     |
| \$L     | a=a+b;b=c}\$  | L -> S;L   |
| \$L;S   | a=a+b;b=c}\$  | S -> V = E |
| \$L;E=V | a=a+b;b=c}\$  | V -> a     |
| \$L;E=a | a=a+b;b=c}\$  | emp(a)     |
| \$L;E=  | =a+b;b=c}\$   | emp(=)     |
| \$L;E   | a+b;b=c}\$    | E -> VO    |
| \$L;OV  | a+b;b=c}\$    | V -> a     |
| \$L;Oa  | a+b;b=c}\$    | emp(a)     |
| \$L;O   | +b;b=c}\$     | O -> +V    |
| \$L;V+  | +b;b=c}\$     | emp(+)     |
| \$L;V   | b;b=c}\$      | V -> b     |
| \$L;b   | b;b=c}\$      | emp(b)     |
| \$L;    | ;b=c}\$       | emp(;)     |
| \$L     | b=c}\$        | L -> S;L   |
| \$L;S   | b=c}\$        | S -> V = E |
| \$L;E=V | b=c}\$        | V -> b     |
| \$L;E=b | b=c}\$        | emp(b)     |
| \$L;E=  | =c}\$         | emp(=)     |
| \$L;E   | c}\$          | E -> VO    |
| \$L,OV  | c}\$          | V -> c     |
| \$L;Oc  | c}\$          | emp(c)     |
| \$L;O   | }\$           | error      |

6. **[1.5 puntos]** Impleméntese parser en Ply

In [61]:
%%file comando.py
import ply.lex as lex
import ply.yacc as yacc

# definir tokens
tokens  = ('DEL', 'ID')

# definir patrones
t_DEL    = r'(. | ;)'
t_ID     = r'[a-zA-Z_][a-zA-Z_0-9]*'
t_ignore = " \t"

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count("\n")
    
def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)

# construir scanner
lexer = lex.lex()

def p_comando(t):
    '''comando : cadena 
               | cadena DEL comando'''

def p_cadena(t):
    '''cadena  : ID
               | ID parametros'''

def p_parametros(t):
    '''parametros : parametro
                  | parametro parametros'''

def p_parametro(t):
    'parametro : ID'

def p_error(t):
    print("Error sintáctico en '%s'" % t.value)

parser = yacc.yacc()
parser.parse('run test1 test2;build main;deploy')

Overwriting comando.py


In [62]:
!python comando.py